In [1]:
!pip install -q -U openai langchain-openai anthropic docling easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 9.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 71.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [63]:
# Version 7, ANTHROPIC

### PARSE PDF

In [21]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
_log = logging.getLogger(__name__)
IMAGE_RESOLUTION_SCALE = 5.0
def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path("/kaggle/input/plumbing/RedHead Rooter Service Call Tree.pdf")
    output_dir = Path("imgs")

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    # Save page images
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.jpeg"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="JPEG")

    # Save images of figures and tables
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-table-{table_counter}.jpeg"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "JPEG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-picture-{picture_counter}.jpeg"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "JPEG")

    end_time = time.time() - start_time

    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")
main()

### GET NODES

In [11]:
import easyocr
reader = easyocr.Reader(['ch_sim','en'], )
nodes = reader.readtext('/kaggle/working/imgs/RedHead Rooter Service Call Tree-1.jpeg',
                        detail = 0, paragraph=True,
                        canvas_size=5000,
                        decoder="wordbeamsearch",
                        ## experiment below. Ref: https://www.jaided.ai/easyocr/documentation/
                        beamWidth = 10,
                        )
nodes

['RedHead Rooter Service Call Tree',
 'CallArrival',
 'RedHead Rooter! How may help yoU today?',
 'Immediately call the on-call technician and provide the Customer contact information 9d address to them_ Follow upas needed。',
 'Customer Service (Past customer or customer with current booking)',
 'Lead',
 'Personal Call',
 'Salesperson',
 '(needs plumbing services)',
 'Angi Call',
 'Calling about UPCOMING booking-',
 'Calling about a PASTICOMPLETED booking-',
 'Collect and log Basic Information in Talkdesk Contact: Name Address Phone Number Email Address How they heard of RedHead Rooter',
 'Does not get personal calls on his business phone',
 "Politely decline and insist you're not interested",
 'Determine the reason fortheir Call',
 'Determine What the customer needs help with',
 'It is urgent / customer is upset',
 'Customer wants to book another appointment',
 'Record the issue and reach out to Shane immediately evenaftechours (be sure to include indicators of urgency)',
 'Verify cus

### LINEARIZE DIAGRAM

In [9]:
import os
os.environ["ANTHROPIC_API_KEY"] = "aaa"
os.environ['OPENAI_API_KEY'] = "aaa"
import anthropic
import base64
from textwrap import dedent

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


image_path = "/kaggle/working/imgs/RedHead Rooter Service Call Tree-1.jpeg"
base64_image = encode_image(image_path)

client = anthropic.Anthropic()

prompt = """The workflow in the provided image is a process diagram that describes steps or nodes that an customer service human agent handles phone calls from a person in different cases.
Based on the example above, your task is to interpret the diagram and generate all various relationships found in it.
Each box means a node.
### Strictly comply with the following rules:
- In the output, include all boxes and texts found in the diagram, and keep the texts as they are. Expand texts are they are in the original diagram.
- In the output, each Step corresponds with a box of any shape (rectangle, haxagon or diamond). Include all the nodes until you find no more.
- In the If-clause, always use "go to Step <step_id>" as the reference to the next step. 
These are extremely important requirements you must follow.

The following elements are nodes or conditions in the diagram. You have to consider them all together with the diagram when creating the output.
### Elements to consider:
['RedHead Rooter Service Call Tree',
 'CallArrival',
 'RedHead Rooter! How may help yoU today?',
 'Immediately call the on-call technician and provide the Customer contact information 9d address to them_ Follow upas needed。',
 'Customer Service (Past customer or customer with current booking)',
 'Lead',
 'Personal Call',
 'Salesperson',
 '(needs plumbing services)',
 'Angi Call',
 'Calling about UPCOMING booking-',
 'Calling about a PASTICOMPLETED booking-',
 'Collect and log Basic Information in Talkdesk Contact: Name Address Phone Number Email Address How they heard of RedHead Rooter',
 'Does not get personal calls on his business phone',
 "Politely decline and insist you're not interested",
 'Determine the reason fortheir Call',
 'Determine What the customer needs help with',
 'It is urgent / customer is upset',
 'Customer wants to book another appointment',
 'Record the issue and reach out to Shane immediately evenaftechours (be sure to include indicators of urgency)',
 'Verify customer is in service range using customer Map',
 'Customer wants to changean upcoming appointment',
 'Customer wants an updated arrival window',
 'Customer wants to cancel upcoming appointment',
 'Customer has a non-urgent question about their service',
 'IN service range',
 'Duringlafter the quoted window',
 'Before the',
 'Appointment is TODAY- Appointment NOT today',
 'quoted window',
 '1) Apologize and say "lam sorry the technician is running behind schedule They Will contact yoU when theyare on the Way 2) sendan SMS to tech to follow up with next Cx',
 'Inform the Customer that their current arrival window is Still the most accurate information。 The tech Will contact them when heis onthe way',
 'Change the appointment and send an SMS the technician immediately to inform them of the same day scheduling changes',
 '1) Cancel the appointment 2) Ask customer Whytheyare cancelling and recordin Call notes 3) Ifsame day Contact Company',
 '1) Record the CUstomers name contact info,and question 2) Send an SMS to the tech Who did the original Service',
 'NOTin service range',
 'Great! Now What Seems to be the problem?',
 'Complete any required follow- Up with the Customer',
 'Refer to Knowledge Base to capture relevant information and verify we can service。',
 'DOES NOT provide this service- Septic pumping (wilIdo other septic)',
 'CAN provide service',
 'Determine if the service is an emergency',
 'It is NOT an emergency',
 'NIfyou are unsure; call the on-call tech to determine ifit is an emegency',
 'Itis an emergency (heeds service ASAP)',
 'Provide set fee pricing for drain cleaning service $125',
 'After hours (disclose there are free estimates, but there is an increased price for after hours appointments)',
 'Regular Ippointment booking',
 'Basic drain cleanings',
 'During business hours',
 'Always say this comes with a free drain camera inspection check up to ensure there are no issues inside the drain',
 'Still wants emergency appointment',
 'Open Housecall Pro and book Customer fornext available appointment time that Customeris also available (If same day inform owner via call)',
 'Callthe on-call tech (schedule found in KB)to confirm availability and timing; follow up with the customeras needed',
 'LOg CUstomer informationin Talkdesk and let the customer Know we are unable to serice them',
 'We are not currently providing call answering during business hours but Would call Shane',
 'Callthe',
 'technician the job is scheduled for immediately after booking to let them know there is a new job scheduled for them today',
 'Same Day Appt。']
"""
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    temperature=0.5,
    max_tokens=4024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": base64_image,
                    },
                },
                {
                    "type": "text",
                    "text": prompt
                }
            ],
        }
    ],
)
print(message.content[0].text)

I'll help break down the service call tree into a structured step-by-step format following the relationships in the diagram.

### Step 1: Call Arrival
- Initial node: "RedHead Rooter! How may help yoU today?"

### Step 2: Branch into call types
If Personal Call:
- Go to Step 3
If Salesperson:
- Go to Step 4
If Lead (needs plumbing services):
- Go to Step 5
If Customer Service (Past customer or customer with current booking):
- Go to Step 6

### Step 3: Personal Call
Action: "Does not get personal calls on his business phone"
End of branch

### Step 4: Salesperson
Action: "Politely decline and insist you're not interested"
End of branch

### Step 5: Lead Processing
1. "Collect and log Basic Information in Talkdesk Contact:
   - Name
   - Address
   - Phone Number
   - Email Address
   - How they heard of RedHead Rooter"
2. "Verify customer is in service range using customer Map"

If IN service range:
- Go to Step 7
If NOT in service range:
- Go to Step 8

### Step 6: Customer Service Br